Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события

- event - событие

    app_install – установка приложения
    app_start – открыть приложения
    register – зарегистрироваться 
    search – перейти на страницу поиска товаров (каталог)
    open_item – открыть товар
    choose_item – отправить товар в корзину
    tap_basket – перейти в корзину
    purchase – подтверждение покупки

- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

    yandex-direct – Яндекс директ
    google_ads – реклама в Google
    vk_ads – реклама в ВК
    instagram_ads – реклама в instagram
    facebook_ads – реклама в facebook
    referal – акция «приведи друга»
    Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  

- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 


In [42]:
import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv('KC_case_data .csv', parse_dates=['date'])

In [44]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   event         object        
 2   purchase_sum  float64       
 3   os_name       object        
 4   device_id     int64         
 5   gender        object        
 6   city          object        
 7   utm_source    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 167.7+ MB


Вычислите MAU февраля

In [46]:
df['month']=df.date.dt.month_name() #для удобства сделаем переменную с названием месяца

In [47]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January


In [7]:
MAU=df.query("month=='February'").device_id.nunique()

In [83]:
print(f'Ответ: MAU февраля {MAU}')

Ответ: MAU февраля 75032


Вычислите количество установок в январе

In [85]:
installs = df.query("month=='January' & event == 'app_install'").device_id.count()

In [88]:
print(f'Ответ: количество установок в январе {installs}')

Ответ: количество установок в январе 80297


Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? 

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [11]:
#сделаем датафрейм с датами установки
install_date = df.query("event == 'app_install'")[['device_id', 'date']].rename(columns={'date': 'install_date'})

In [12]:
install_date.head()

,device_id,install_date
2515061,4921563,2020-01-01
2515062,1311583,2020-01-01
2515063,23006325,2020-01-01
2515064,5645652,2020-01-01
2515065,8798567,2020-01-01


In [13]:
#присоединим к изначальному датафрейму
full_data = pd.merge(df, install_date, on='device_id', how='left')

In [14]:
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January,NaT
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January,2020-01-01


In [15]:
#сделаем то же самое для даты покупки
purchase_date=df.query("event=='purchase'")[['device_id', 'date']].rename(columns={'date': 'purchase_date'})

In [16]:
full_data = pd.merge(full_data, purchase_date, on='device_id', how='left')

In [17]:
#теперь у нас есть все, чтобы вычислить разницу во времени между установкой и покупкой
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date,purchase_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,2020-01-09
2,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-01
3,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-11
4,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-16


In [18]:
full_data['timediff'] = (full_data['purchase_date'] - full_data['install_date']).dt.days

In [19]:
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date,purchase_date,timediff
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,2020-01-07,6.0
1,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January,2020-01-01,2020-01-09,8.0
2,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-01,0.0
3,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-11,10.0
4,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January,2020-01-01,2020-01-16,15.0


In [20]:
#отберем в отдельный датафрейм пользователей, оформивших покупку в течение недели
seven_days_purchase = full_data.query("timediff<=7 & event=='purchase'")

In [21]:
#посчитаем конверсию на основании датафрейма с установками
#чтобы узнать id для покупок, посчитаем число id из датафрейма с покупками
cohort_conversion = install_date.groupby('install_date').agg(
    total_installs=('device_id', 'nunique'),
    total_purchases=('device_id', lambda x: x.isin(seven_days_purchase['device_id']).sum())
).reset_index()

In [22]:
cohort_conversion['conversion_rate'] = cohort_conversion['total_purchases'] / cohort_conversion['total_installs']

In [89]:
cohort_sorted = cohort_conversion.sort_values('conversion_rate',ascending = False)

In [91]:
print(f'Ответ: конверсия была наибольшей для {cohort_sorted.install_date[0].date()}')

Ответ: конверсия была наибольшей для 2020-01-01


С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [115]:
channel = df.groupby('utm_source', as_index = False).device_id.nunique()\
.sort_values('device_id', ascending = False).reset_index(drop= True)

In [116]:
channel = channel.query("utm_source != '-'")

In [117]:
channel

,utm_source,device_id
1,yandex-direct,40712
2,google_ads,38096
3,vk_ads,34086
4,instagram_ads,31048
5,facebook_ads,25959
6,referal,15926


In [119]:
print(f'Ответ: Наибольшее число пользователей пришло с  {channel.utm_source[1]}')

Ответ: Наибольшее число пользователей пришло с  yandex-direct


Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [25]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,January
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,January
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,January
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,January
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,January


In [120]:
median_check = df.query("event == 'purchase'")[['device_id', 'date','purchase_sum','utm_source']]\
.groupby('device_id', as_index = False)\
.agg(first_date = ('date', 'min'), first_check = ('purchase_sum', 'first'), utm_source = ('utm_source','first'))\
.groupby('utm_source', as_index = False).agg(median_check=('first_check','median'))\
.sort_values('median_check', ascending = False).reset_index(drop=True)
median_check = median_check.query("utm_source != '-'")

In [121]:
median_check

,utm_source,median_check
1,referal,395.5
2,instagram_ads,393.5
3,vk_ads,393.0
4,yandex-direct,392.5
5,google_ads,390.5
6,facebook_ads,389.0


In [122]:
print(f'Ответ: Наибольший медианный чек у пользователей, пришедших с канала {median_check.utm_source[1]}')

Ответ: Наибольший медианный чек у пользователей, пришедших с канала referal


Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

Подсказка: 

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [49]:
#Отберем пользователей, зашедших в приложение и совершивших покупку
all_users=full_data.query("event=='app_start'")
purchases=full_data.query("event=='purchase'")


In [52]:
#Создадим датафрейм, в котором отберем первые покупки
first_purchases = purchases.groupby('device_id', as_index = False).apply(lambda x: x.sort_values('date'))

first_purchases = first_purchases.groupby('device_id').head(1)

In [54]:
#То же самое сделаем для первого открытия приложения
first_opened = all_users.groupby('device_id', as_index = False).apply(lambda x: x.sort_values('date'))

first_opened = all_users.groupby('device_id').head(1)


In [58]:
#Найдем число уникальных пользователей, которые совершили первую покупку
first_purchase_users = first_purchases.groupby('utm_source', as_index = False).device_id.nunique().rename(columns = {'device_id':'purchases'})

#И впервые открыли приложение
first_opened_users = first_opened.groupby('utm_source', as_index = False).device_id.nunique().rename(columns = {'device_id':'opened'})



In [129]:
#Вычислим CR
CR = pd.merge(first_opened_users, first_purchase_users, on = 'utm_source', how = 'left')
CR['CR'] = CR['purchases'] / CR['opened']
CR = CR.query("utm_source != '-'").sort_values('CR').reset_index(drop = True)
CR

,utm_source,opened,purchases,CR
0,yandex-direct,34441,10936,0.317529
1,google_ads,31437,10167,0.323409
2,instagram_ads,24818,9820,0.395681
3,vk_ads,27905,11460,0.410679
4,facebook_ads,18844,7903,0.419391
5,referal,11983,5803,0.484269


In [128]:
print(f'Ответ: Самая низкая конверсия в первую покупку у пользователей, пришедших с {CR.utm_source[0]}')

Ответ: Самая низкая конверсия в первую покупку у пользователей, пришедших с yandex-direct


Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

In [66]:
#ROMI - (валовая прибыль-маркетинговые расходы) / маркетинговые расходы
#Валовая прибыль (gross profit)  — выручка с вычетом себестоимости проданного товара или услуги
#Но у нас данных о себестоимости нет, поэтому считаем так
#Начнем с того, что посчитаем для каждого источника число пользователей и выручку (себестоимость не дана, поэтому считаем без нее)
ROMI = full_data.groupby('utm_source', as_index = False).agg(profit = ('purchase_sum','sum'), clients = ('device_id','nunique'))

In [72]:
ROMI

,utm_source,profit,clients
0,-,108901736.5,52273
1,facebook_ads,60668237.5,25959
2,google_ads,51002137.0,38096
3,instagram_ads,68598287.5,31048
4,referal,45357215.5,15926
5,vk_ads,73764243.0,34086
6,yandex-direct,58175485.5,40712


In [76]:
#Напишем функцию, которая заполнит переменную с затратами, в зависимости от источника

def costs (x):
    if x['utm_source'] == 'facebook_ads':
        return 8590498
    elif x['utm_source'] == 'google_ads':
        return 10534878
    elif x['utm_source'] == 'instagram_ads':
        return 8561626
    elif x['utm_source'] == 'vk_ads':
        return 9553531
    elif x['utm_source'] == 'yandex-direct':
        return 10491707
    elif x['utm_source'] == 'referal':
        return x['clients'] * 200

In [77]:
ROMI['advertising_costs'] = ROMI.apply(costs, axis = 1)

In [79]:
#Остается посчитать ROMI
ROMI['ROMI'] = (ROMI['profit'] - ROMI['advertising_costs']) - ROMI['advertising_costs']

In [132]:
ROMI = ROMI.query("utm_source != '-'").sort_values('ROMI', ascending = False).reset_index(drop = True)

In [133]:
ROMI

,utm_source,profit,clients,advertising_costs,ROMI
0,vk_ads,73764243.0,34086,9553531.0,54657181.0
1,instagram_ads,68598287.5,31048,8561626.0,51475035.5
2,facebook_ads,60668237.5,25959,8590498.0,43487241.5
3,referal,45357215.5,15926,3185200.0,38986815.5
4,yandex-direct,58175485.5,40712,10491707.0,37192071.5
5,google_ads,51002137.0,38096,10534878.0,29932381.0


In [134]:
print(f'Ответ: Самый низкий ROMI у {ROMI.utm_source[0]}')

Ответ: Самый низкий ROMI у vk_ads
